In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

100.1%

Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw


113.5%

Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw


100.4%

Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw


180.4%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw
Processing...
Done!


D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

# 装载训练集
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [4]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(784,500),nn.Dropout(p=0.5),nn.Tanh())#注意：dropout有时候用效果好有时候用效果不好
        self.layer2 = nn.Sequential(nn.Linear(500,300),nn.Dropout(p=0.5),nn.Tanh())#p表示百分之多少的神经元不工作
        self.layer3 = nn.Sequential(nn.Linear(300,10),nn.Softmax(dim=1))    
    def forward(self,x):
        # ([64, 1, 28, 28])->(64,784)
        x = x.view(x.size()[0], -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [6]:
LR = 0.5
# 定义模型
model = Net()
# 定义代价函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.SGD(model.parameters(), LR)

In [9]:
def train():
    model.train()#定义模型训练状态，此时dropout起作用
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()#定义模型测试状态，此时dropout不起作用
    correct = 0
    for i,data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    
    #训练集的准确率
    correct = 0
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item()/len(train_dataset)))

In [11]:
for epoch in range(20):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test acc:0.9636
Train acc:0.9708
epoch: 1
Test acc:0.9656
Train acc:0.9735333333333334
epoch: 2
Test acc:0.9677
Train acc:0.9755166666666667
epoch: 3
Test acc:0.9682
Train acc:0.97575
epoch: 4
Test acc:0.9675
Train acc:0.9752166666666666
epoch: 5
Test acc:0.9705
Train acc:0.9764
epoch: 6
Test acc:0.9692
Train acc:0.9765333333333334
epoch: 7
Test acc:0.9691
Train acc:0.9771333333333333
epoch: 8
Test acc:0.9703
Train acc:0.9780666666666666
epoch: 9
Test acc:0.9708
Train acc:0.9794333333333334
epoch: 10
Test acc:0.9706
Train acc:0.9799
epoch: 11
Test acc:0.9717
Train acc:0.9796166666666667
epoch: 12
Test acc:0.9698
Train acc:0.979
epoch: 13
Test acc:0.9719
Train acc:0.9805833333333334
epoch: 14
Test acc:0.9711
Train acc:0.9813333333333333
epoch: 15
Test acc:0.9713
Train acc:0.9800666666666666
epoch: 16
Test acc:0.9726
Train acc:0.9814
epoch: 17
Test acc:0.971
Train acc:0.9805333333333334
epoch: 18
Test acc:0.9711
Train acc:0.9807833333333333
epoch: 19
Test acc:0.973
Train acc:0.9

In [ ]:
epoch: 0
Test acc:0.8883
epoch: 1
Test acc:0.9008
epoch: 2
Test acc:0.9088
epoch: 3
Test acc:0.9128
epoch: 4
Test acc:0.9138
epoch: 5
Test acc:0.9153
epoch: 6
Test acc:0.916
epoch: 7
Test acc:0.9169
epoch: 8
Test acc:0.9184
epoch: 9
Test acc:0.92